<h1 style="text-align:center; text-decoration:underline">Nice to MET you</h1>
<p style="text-align:center"><b>More than 500 Italian paintings are now exhibited at Metropolitan Museum of Art, New York. What's the history behind their journeys?</b></p>
<hr>

<p style="text-align: justify"><b>Nice to MET you</b> is an Information Visualization project, exploring the Italian paintings that are exhibited at <b><em>The Metropolitan Museum of Art</em></b> in New York. More than five hundred Italian masterpieces are currently preserved at the MET, but how did they get there? The aim of this project is to examine the provenance of these artworks and those actors who contributed to the growth of this collection.</p>

<p style="text-align: justify">The project relies on two major sources, whose specific contribution will be clarified in the next section: The MET .csv Dataset (available at this <a href="https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv">link</a>) and the <a href="http://data.fondazionezeri.unibo.it/">Zeri &amp; Lode RDF dataset</a>.</p>

<p style="text-align: justify">This Jupyter Notebook is intended to provide readers with a documentation of the project.</p> 

<h2 id="#anal">Data Analysis</h2>

<p style="text-align:justify">As part of the <b>Open Access Initiave</b>, <b><em>The Metropolitan Museum of Art</em></b> provides select datasets of information on more than 470,000 artworks. Among them, one can observe a considerable number of <b>Italian paintings</b>, an initial cataloguing of which was carried out by the Italian connoisseur <b>Federico Zeri</b> (1971-1986). The Metropolitan dataset contains many details about the creation and the accession of each artwork, along with cataloguing information and URI references to <b>Wikidata</b> and the Getty <b>Art &amp; Architecture Thesaurus</b>.</p>
<p style="text-align:justify">Nevertheless, some data about the provenance are still missing: this is the reason why, the present project exploits the information made available by the <b>Zeri &amp; Lode</b> project. The first release of the Zeri Photo Archive RDF dataset (April, 2016) represents a considerable subset of data already available at the <a href="http://catalogo.fondazionezeri.unibo.it/cerca/opera">Zeri Catalog web site</a>. Not all the Italian paintings preserved at the MET are currently represented in the RDF graph. However, the Zeri &amp; Lode project presents a major advantage: it allows users to better examine the history of the artworks. Each available transaction/transfer is in fact associated with its own URI, which is directly linked to the URIs of both the surrender and the receiver. In other words, the MET dataset only focuses on the last transfer of the artwork (column "Credit Line"; e.g.: <em>Bequest of Harry G. Sperling, 1971</em>), while the Zeri Catalog provides some more information also about the previous transactions (e.g.: <a href="http://data.fondazionezeri.unibo.it/id/artwork/16633/provenance-4" target="blank">http://data.fondazionezeri.unibo.it/id/artwork/16633/provenance-4</a>), allowing users to better track down some of the artworks' trajectories.</p>

<b style="font-size: 22px; text-decoration:underline">Step 1: Import Libraries</b>

In [1]:
#pandas
from pandas import read_sql
import pandas as pd

#sql 
from sqlite3 import connect

#sparql
from SPARQLWrapper import SPARQLWrapper, JSON
import sparql_dataframe

#regular expressions
import re

<b style="font-size: 22px; text-decoration:underline">Step 2: Extract Italian Paintaings from the general MET dataset</b>

<p>The MET dataset (<b>.csv</b> format) is converted into a <b>DataFrame</b>. The columns "Artist Nationality"/"Artist Display Bio" and "Department" allow us to isolate Italian Paintings creating a smaller DataFrame (<b><code>italianDataset</code></b>). This task is carried out through an <b>SQL</b> query.</p>

In [2]:
csvData = pd.read_csv("https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv", encoding='utf-8')
with connect("metDB.db") as con:
    csvData.to_sql("MetCatalog", con, if_exists="replace", index=False)
    con.commit()
with connect("metDB.db") as con:
    query = """
        SELECT *
        FROM `MetCatalog`
        WHERE ("Artist Nationality" LIKE '%talian%' OR "Artist Display Bio" LIKE '%talian%') AND "Department" = "European Paintings";
    """
    italianDataset = pd.read_sql(query, con)
italianDataset.info()

C:\Users\sebas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,7,10,11,12,13,14,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\sebas\anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532 entries, 0 to 531
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Object Number            532 non-null    object
 1   Is Highlight             532 non-null    int64 
 2   Is Timeline Work         532 non-null    int64 
 3   Is Public Domain         532 non-null    int64 
 4   Object ID                532 non-null    int64 
 5   Gallery Number           230 non-null    object
 6   Department               532 non-null    object
 7   AccessionYear            532 non-null    object
 8   Object Name              531 non-null    object
 9   Title                    532 non-null    object
 10  Culture                  0 non-null      object
 11  Period                   0 non-null      object
 12  Dynasty                  0 non-null      object
 13  Reign                    0 non-null      object
 14  Portfolio                0 non-null      o

<b style="font-size: 22px; text-decoration:underline">Step 3: Retrieve all the artworks currently preserved at the MET (Zeri Catalog)</b>

<p>The <b>Federico Zeri Catalog</b> deals with a huge collection of artworks: some of them are included in the MET dataset, but the Zeri Catalog offers a much more detailed presentation about the provenance of some paintings. In this case, we need to perform a <b>SPARQL</b> query: thus, we need an <b>endpoint</b> that allows us to reach the RDF dataset.</p>
<p>The CIDOC Conceptual Reference Model (<b>CIDOC-CRM</b>) was used to represent the richness of Zeri Photo Archive. Thus, <code><b>crm:</b></code> is the main PREFIX for this query.</p>
<p><a href="https://w3id.org/zericatalog/the-metropolitan-museum-of-art-new-york-ny">&lt;https://w3id.org/zericatalog/the-metropolitan-museum-of-art-new-york-ny&gt;</a> is the URIRef for the Metropolitan Museum.</p> 

In [3]:
endpoint = "http://data.fondazionezeri.unibo.it/sparql"

query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
SELECT DISTINCT ?artwork ?label_artwork ?inventory
WHERE {
  ?artwork crm:P50_has_current_keeper <https://w3id.org/zericatalog/the-metropolitan-museum-of-art-new-york-ny> .
  ?artwork  rdfs:label ?label_artwork.
  ?artwork crm:P55_has_current_location ?location .
  ?location  crm:P3_has_note ?inventory . 
  FILTER(LANG(?label_artwork) = "" || LANGMATCHES(LANG(?label_artwork), "en"))

}
"""

df = sparql_dataframe.get(endpoint, query, post=True)
df

,artwork,label_artwork,inventory
0,https://w3id.org/zericatalog/artwork/18789/item,"Artwork ""Luciani Sebastiano , Ritratto di Cris...",inv. 00.18.2
1,https://w3id.org/zericatalog/artwork/16633/item,"Artwork ""Del Conte Jacopino , Sacra Famiglia"" ...","inv. 1976.100.15, lascito Harry G. Sperling"
2,https://w3id.org/zericatalog/artwork/18917/item,"Artwork ""Anonimo sec. XVI , Ritratto di monaco...",inv. 1986.339.1
3,https://w3id.org/zericatalog/artwork/18008/item,"Artwork ""Rossi Francesco de' , Ritratto maschi...",inv. 45.128.11
4,https://w3id.org/zericatalog/artwork/18022/item,"Artwork ""Rossi Francesco de' , Ritratto maschi...",inv. 55.14
...,...,...,...
103,https://w3id.org/zericatalog/artwork/50670/item,"Artwork ""Bonvicino Alessandro , Ritratto di ge...",inv. 28.79
104,https://w3id.org/zericatalog/artwork/49125/item,"Artwork ""Tisi Benvenuto , San Nicola da Tolent...",inv. 17.190.24
105,https://w3id.org/zericatalog/artwork/49129/item,"Artwork ""Tisi Benvenuto , San Nicola da Tolent...",inv. 17.190.23
106,https://w3id.org/zericatalog/artwork/50819/item,"Artwork ""Bonvicino Alessandro , Deposizione di...",inv. 12.61


<b style="font-size: 22px; text-decoration:underline">Step 4: Retrieve all data about about each artwork's transfers (Zeri Catalog)</b>

<p>We are now interested in discovering something more about the artworks' provenance:</p>
<ul>
    <li>history: the number of the artwork's transfers recorded in the Zeri Dataset</li>
    <li>receivers: the adressees of the trasnfers</li>
    <li>surrenders: the surrenders of the artwork's custody</li>
</ul>

In [4]:
#Works' History - Federico Zeri Catalog

URI_string = ""

for index, row in df.iterrows():
    if len(URI_string) == 0:
        URI_string = "?artwork = <" + row["artwork"] + ">"
    else: 
        URI_string = URI_string + "||" + "?artwork = <" + row["artwork"] + ">"

final_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
SELECT ?artwork  (count(distinct ?tras) AS ?history) (group_concat(distinct ?label_receiver ; separator = "; ") AS ?receivers) (group_concat(distinct ?inventory ; separator = "; ") AS ?id) (group_concat(distinct ?label_surrender ; separator = "; ") AS ?surrenders)          
WHERE {{
  ?trasferimento_global crm:P30_transferred_custody_of ?artwork .
  ?tras crm:P30_transferred_custody_of ?artwork .
  OPTIONAL {{ ?trasferimento_global crm:P28_custody_surrendered_by ?surrender }} . 
  ?receiver crm:P29i_received_custody_through ?trasferimento_global .
  ?artwork crm:P50_has_current_keeper <https://w3id.org/zericatalog/the-metropolitan-museum-of-art-new-york-ny> .
  ?artwork crm:P55_has_current_location ?location .
  ?location  crm:P3_has_note ?inventory .
  FILTER ({0})
  
  ?artwork  rdfs:label ?label_artwork.
  ?surrender rdfs:label ?label_surrender.
  ?receiver rdfs:label ?label_receiver.
  FILTER(?receiver != <https://w3id.org/zericatalog/>)
  FILTER(?surrender != <https://w3id.org/zericatalog/>)
  
  FILTER(LANG(?label_artwork) = "" || LANGMATCHES(LANG(?label_artwork), "en"))
  FILTER(LANG(?label_surrender) = "" || LANGMATCHES(LANG(?label_surrender), "en"))
  FILTER(LANG(?label_receiver) = "" || LANGMATCHES(LANG(?label_receiver), "en"))
}} GROUP BY ?artwork
"""

final_df = sparql_dataframe.get(endpoint, final_query.format(URI_string), post=True)
final_df

,artwork,history,receivers,id,surrenders
0,https://w3id.org/zericatalog/artwork/39117/item,6,The Metropolitan Museum of Art; Collezione G. ...,inv. 1970.134.1,"Asta Sotheby's; Ashburnham Place, collezione A..."
1,https://w3id.org/zericatalog/artwork/39828/item,5,The Metropolitan Museum of Art; E. Sestieri; C...,inv. 48.17.12,E. Sestieri; Collezione Malanca; Collezione G....
2,https://w3id.org/zericatalog/artwork/39818/item,5,The Metropolitan Museum of Art; E. Sestieri; C...,inv. 48.17.5,E. Sestieri; Collezione Malanca; Collezione G....
3,https://w3id.org/zericatalog/artwork/45055/item,1,The Metropolitan Museum of Art,"inv. 14.40.650, dono B. Altman",Collezione B. Altman
4,https://w3id.org/zericatalog/artwork/39821/item,5,The Metropolitan Museum of Art; E. Sestieri; C...,inv. 48.17.7,E. Sestieri; Collezione Malanca; Collezione G....
5,https://w3id.org/zericatalog/artwork/39831/item,5,The Metropolitan Museum of Art; E. Sestieri; C...,inv. 48.17.14,E. Sestieri; Collezione Malanca; Collezione G....
6,https://w3id.org/zericatalog/artwork/16633/item,4,The Metropolitan Museum of Art; Collezione H.G...,"inv. 1976.100.15, lascito Harry G. Sperling",Collezione H.G. Sperling; Kleinberger Gallerie...
7,https://w3id.org/zericatalog/artwork/41423/item,2,The Metropolitan Museum of Art; Asta Christie's,inv. 1973.311.1,"Asta Christie's; Harewood House, collezione Ea..."
8,https://w3id.org/zericatalog/artwork/60085/item,1,The Metropolitan Museum of Art,inv. 1971.115.3b,"Duveen Brothers, Inc."
9,https://w3id.org/zericatalog/artwork/39841/item,5,The Metropolitan Museum of Art; E. Sestieri; C...,inv. 48.17.20,E. Sestieri; Collezione Malanca; Collezione G....


<b style="font-size: 22px; text-decoration:underline">Step 5: Merge multiple ids related to the same artwork</b>

<p style="text-align:justify">Our goal is to combine the two dataframes we have created so far (<code><b>italiaDataset</b></code> and <code><b>final_df</b></code>), thus, we need to align the artworks according to their Object Number (the only element shared by the two original Datasets). We then need to manipulate the column "id" (later renamed as "Object Number") of the <code><b>final_df</b></code>. First, the abbreviation "inv." at the beginning of each id must be removed, so that we can effectively compare the cataloguing number. Moreover, in the Zeri RDF dataset, multiple artworks related to the same group are considered as individual, while the MET dataset considers them as a single conceptual object. Therefore, it is necessary to group the artworks' ids from the Zeri dataset.</p>

In [5]:
a_list = []

for idx, row in final_df.iterrows():
    a_list.append(str(row["id"]).replace("inv. ", "").split(",")[0])
a_list
final_df["Object Number"] = a_list
final_df = final_df.drop(['id'], axis=1)


ser = final_df["Object Number"]
ser = ser.tolist()

id_dict = {}
a_list = []

for el in ser:
    w = str()
    for inn in el.split(".")[:-1]:
        if len(w) > 0:
            w = w + "." + inn
        else:
            w = w + inn
    if w not in id_dict:
        id_dict[w] = [el]
    else:
        a = id_dict[w]
        a.append(el)
        id_dict[w] = a

artwork_uris = []

number_rows = final_df.shape[0]

for el in id_dict["48.17"]:
    for idx, row in final_df.iterrows():
        if el == row["Object Number"]:
            history = row["history"]
            receivers = row["receivers"]
            surrenders = row["surrenders"]
            artwork_uris.append(row["artwork"])
            final_df = final_df.drop([idx])

final_df.loc[number_rows, :] = [(", ".join(artwork_uris)), history, receivers, surrenders, "48.17.1–22"]
number_rows = number_rows + 1

artwork_uris = []
for el in id_dict["1971.115"]:
    for idx, row in final_df.iterrows():
        if el == row["Object Number"]:
            history = row["history"]
            receivers = row["receivers"]
            surrenders = row["surrenders"]
            artwork_uris.append(row["artwork"])
            final_df = final_df.drop([idx])
            
final_df.loc[number_rows, :] = [(", ".join(artwork_uris)), history, receivers, surrenders, "1971.115.3ab"]
final_df

,artwork,history,receivers,surrenders,Object Number
0,https://w3id.org/zericatalog/artwork/39117/item,6.0,The Metropolitan Museum of Art; Collezione G. ...,"Asta Sotheby's; Ashburnham Place, collezione A...",1970.134.1
3,https://w3id.org/zericatalog/artwork/45055/item,1.0,The Metropolitan Museum of Art,Collezione B. Altman,14.40.650
6,https://w3id.org/zericatalog/artwork/16633/item,4.0,The Metropolitan Museum of Art; Collezione H.G...,Collezione H.G. Sperling; Kleinberger Gallerie...,1976.100.15
7,https://w3id.org/zericatalog/artwork/41423/item,2.0,The Metropolitan Museum of Art; Asta Christie's,"Asta Christie's; Harewood House, collezione Ea...",1973.311.1
10,https://w3id.org/zericatalog/artwork/32096/item,4.0,The Metropolitan Museum of Art; P. Corsini; Co...,P. Corsini; Collezione S. Brunswick; Hammer Ga...,1984.74
11,https://w3id.org/zericatalog/artwork/45062/item,4.0,The Metropolitan Museum of Art; Collezione J. ...,Collezione J. Pierpont Morgan; Collezioni real...,16.30ab
12,https://w3id.org/zericatalog/artwork/46494/item,1.0,The Metropolitan Museum of Art,Collezione G. e F. Blumenthal,41.100.12
13,https://w3id.org/zericatalog/artwork/18022/item,3.0,The Metropolitan Museum of Art; Collezione J.H...,Collezione J.H. Harding; Asta Christie's; Coll...,55.14
18,https://w3id.org/zericatalog/artwork/36357/item,3.0,The Metropolitan Museum of Art; Collezione T.M...,Collezione T.M. Davis; Collezione Baude; Colle...,30.95.270
21,https://w3id.org/zericatalog/artwork/45063/item,1.0,The Metropolitan Museum of Art,Convento di S. Antonio da Padova,32


<b style="font-size: 22px; text-decoration:underline">Step 6: Merge the Dataframes</b>

<p>We are now ready to merge our Dataframes (<code><b>italiaDataset</b></code> and <code><b>final_df</b></code>). In this way, the data from the two datasets (MET and Zeri Catalog) will be aligned in a unique Dataframe.</p>

In [6]:
#merging MET and Federico Zeri

from pandas import merge


result = pd.merge(final_df, italianDataset)
result.rename(columns={"Artist Alpha Sort":"ArtistAlphaSort"},inplace=True)
result.to_csv("data_management/result.csv")
result

,artwork,history,receivers,surrenders,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,...,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,https://w3id.org/zericatalog/artwork/39117/item,6.0,The Metropolitan Museum of Art; Collezione G. ...,"Asta Sotheby's; Ashburnham Place, collezione A...",1970.134.1,0,0,1,436567,None,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911541,None,"Metropolitan Museum of Art, New York, NY",Infants|Women|Saint John the Baptist|Bedrooms|...,http://vocab.getty.edu/page/aat/300189561|http...,https://www.wikidata.org/wiki/Q998|https://www...
1,https://w3id.org/zericatalog/artwork/45055/item,1.0,The Metropolitan Museum of Art,Collezione B. Altman,14.40.650,0,1,1,437823,608.0,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19904807,None,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
2,https://w3id.org/zericatalog/artwork/16633/item,4.0,The Metropolitan Museum of Art; Collezione H.G...,Collezione H.G. Sperling; Kleinberger Gallerie...,1976.100.15,0,0,1,436770,None,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912153,None,"Metropolitan Museum of Art, New York, NY",Holy Family|Saint Joseph|Christ|Virgin Mary,http://vocab.getty.edu/page/ia/901001175|http:...,https://www.wikidata.org/wiki/Q618057|https://...
3,https://w3id.org/zericatalog/artwork/41423/item,2.0,The Metropolitan Museum of Art; Asta Christie's,"Asta Christie's; Harewood House, collezione Ea...",1973.311.1,0,1,1,435722,626.0,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905185,None,"Metropolitan Museum of Art, New York, NY",Armor|Boys|Men|Portraits,http://vocab.getty.edu/page/aat/300226591|http...,https://www.wikidata.org/wiki/Q20793164|https:...
4,https://w3id.org/zericatalog/artwork/32096/item,4.0,The Metropolitan Museum of Art; P. Corsini; Co...,P. Corsini; Collezione S. Brunswick; Hammer Ga...,1984.74,0,1,1,437342,None,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911628,None,"Metropolitan Museum of Art, New York, NY",Sadness|Suffering|Mary Magdalene|Saint John th...,http://vocab.getty.edu/page/aat/300055166|http...,https://www.wikidata.org/wiki/Q169251|https://...
5,https://w3id.org/zericatalog/artwork/45062/item,4.0,The Metropolitan Museum of Art; Collezione J. ...,Collezione J. Pierpont Morgan; Collezioni real...,16.30ab,1,1,1,437372,962.0,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q27033728,None,"Metropolitan Museum of Art, New York, NY",Madonna and Child|Saint Catherine|Saint John t...,http://vocab.getty.edu/page/ia/901000052| |htt...,https://www.wikidata.org/wiki/Q9309699| |https...
6,https://w3id.org/zericatalog/artwork/46494/item,1.0,The Metropolitan Museum of Art,Collezione G. e F. Blumenthal,41.100.12,0,1,1,437822,608.0,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905075,None,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
7,https://w3id.org/zericatalog/artwork/18022/item,3.0,The Metropolitan Museum of Art; Collezione J.H...,Collezione J.H. Harding; Asta Christie's; Coll...,55.14,0,1,1,437598,None,...,None,Paintings,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912783,None,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
8,ht

<b style="font-size: 22px; text-decoration:underline">Step 7: retrieve further details about the entities involved in the artworks' cusotdy</b>

<p style="text-align:justify">The Zeri Dataset presents further details about some custody periods during which a certain artwork was kept by a specific entity. It is then possible to retrieve the time-span and the location of the custody through a SPARQL query. The last column of the new DataFrame ("location"), contains useful references to controlled knowledge bases/datasets.</p>

In [7]:
new_string = ""
for idx, row in result.iterrows():
    if idx == 0:
        if "," not in row["artwork"]:
            new_string = new_string + "strstarts(str(?custody), '" + row["artwork"].replace("/item", "/custody") + "')"
        else:
            artw = row["artwork"].split(",")[0]
            new_string = new_string + "strstarts(str(?custody), '" + artw.replace("/item", "/custody") + "')"
    else:
        if "," not in row["artwork"]:
            new_string = new_string + "|| strstarts(str(?custody), '" + row["artwork"].replace("/item", "/custody") + "')"
        else: 
            artw = row["artwork"].split(",")[0]
            new_string = new_string + "|| strstarts(str(?custody), '" + artw.replace("/item", "/custody") + "')"
access = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?custody ?time_span ?start_date ?end_date ?place_label ?label_receiver ?location
WHERE {{
  ?receiver crm:P29i_received_custody_through ?custody .
  ?receiver crm:P74_has_current_or_former_residence ?place .
  ?custody crm:P4_has_time_span ?time_span .
  ?time_span ti:hasIntervalEndDate ?end_date .
  ?time_span ti:hasIntervalStartDate ?start_date .
  ?place owl:sameAs ?location .
  ?place rdfs:label ?place_label .
  ?receiver rdfs:label ?label_receiver .
  FILTER(LANG(?place_label) = "" || LANGMATCHES(LANG(?place_label), "en"))
  FILTER(LANG(?label_receiver) = "" || LANGMATCHES(LANG(?label_receiver), "en"))
  FILTER ({0})
}}
"""
third_df = sparql_dataframe.get(endpoint, access.format(new_string), post=True)   
third_df.to_csv("data_management/final.csv")

third_df

,custody,time_span,start_date,end_date,place_label,label_receiver,location
0,https://w3id.org/zericatalog/artwork/16633/cus...,https://w3id.org/zericatalog/date/1949-1958,1949-01-01T00:00:00Z,1958-01-01T00:00:00Z,Parigi,Collezione O. Wertheimer,http://sws.geonames.org/2988507
1,https://w3id.org/zericatalog/artwork/16633/cus...,https://w3id.org/zericatalog/date/1949-1958,1949-01-01T00:00:00Z,1958-01-01T00:00:00Z,Parigi,Collezione O. Wertheimer,http://www.wikidata.org/entity/Q90
2,https://w3id.org/zericatalog/artwork/16633/cus...,https://w3id.org/zericatalog/date/1949-1958,1949-01-01T00:00:00Z,1958-01-01T00:00:00Z,Parigi,Collezione O. Wertheimer,http://dbpedia.org/resource/Paris
3,https://w3id.org/zericatalog/artwork/16692/cus...,https://w3id.org/zericatalog/date/1934-1977,1934-01-01T00:00:00Z,1977-01-01T00:00:00Z,Parigi,Collezione Dillon,http://sws.geonames.org/2988507
4,https://w3id.org/zericatalog/artwork/16692/cus...,https://w3id.org/zericatalog/date/1934-1977,1934-01-01T00:00:00Z,1977-01-01T00:00:00Z,Parigi,Collezione Dillon,http://www.wikidata.org/entity/Q90
5,https://w3id.org/zericatalog/artwork/16692/cus...,https://w3id.org/zericatalog/date/1934-1977,1934-01-01T00:00:00Z,1977-01-01T00:00:00Z,Parigi,Collezione Dillon,http://dbpedia.org/resource/Paris
6,https://w3id.org/zericatalog/artwork/16692/cus...,https://w3id.org/zericatalog/date/1836-1934,1836-01-01T00:00:00Z,1934-01-01T00:00:00Z,Parigi,Collezione Chaix-d'Est Anges,http://sws.geonames.org/2988507
7,https://w3id.org/zericatalog/artwork/16692/cus...,https://w3id.org/zericatalog/date/1836-1934,1836-01-01T00:00:00Z,1934-01-01T00:00:00Z,Parigi,Collezione Chaix-d'Est Anges,http://www.wikidata.org/entity/Q90
8,https://w3id.org/zericatalog/artwork/16692/cus...,https://w3id.org/zericatalog/date/1836-1934,1836-01-01T00:00:00Z,1934-01-01T00:00:00Z,Parigi,Collezione Chaix-d'Est Anges,http://dbpedia.org/resource/Paris
9,https://w3id.org/zericatalog/artwork/33068/cus...,https://w3id.org/zericatalog/date/1925-1928,1925-01-01T00:00:00Z,1928-01-01T00:00:00Z,Londra,"Duveen Brothers, Inc.",https://w3id.org/zericatalog/place/london


<b style="font-size: 22px; text-decoration:underline">Step 8: split the Credit Line</b>

<p style="text-align:justify">As last step, we need to make available the content of the <b>Credit Line</b> column. All the information about how the MET acquired each artwork is included in this column. Though we have already extracted many details from the Zeri Catalog, we haven't focused yet on those artworks that are not included in the RDF Dataset. Consequently, we would like to retrieve as many data as possible from the MET Dataset.</p>
<p style="text-align:justify">In particular, we are interested in two major aspects: how the MET acquired/received the custody of the artwork (Purchase, Bequest, Collection, Gift are the possible labels for <b>"last Transfer"</b>) and the entities involved in this last transition (<b>"agentsTransfer"</b>). Unfortunately, no detail is available about previous transfers.</p>

In [8]:
empty = pd.Series([""])
resp = pd.Series([""])

def add_last_transfer(num, obj):
    italianDataset.at[num, "lastTransfer"] = obj

def add_agent(num, agent):
    if agent.startswith("and "):
        agent = agent.replace("and ", "")
    if agent.endswith(" "):
        agent = agent[:-1]
    if type(italianDataset.at[num, "agentsTransfer"]) == type("") and len(italianDataset.at[num, "agentsTransfer"]) > 0:
        if agent not in italianDataset.at[num, "agentsTransfer"].split(", ") and (agent + " Fund") not in italianDataset.at[num, "agentsTransfer"].split(", "):
            italianDataset.at[num, "agentsTransfer"] = italianDataset.at[num, "agentsTransfer"] + ", " + agent
        else:
            return None
    else:
        italianDataset.at[num, "agentsTransfer"] = agent

def and_case(a_string, credit):
    if " and " in credit and "Mr" not in credit and "Mrs." not in credit and "Foundation" not in credit and "Fund" not in credit and "Collection" not in credit and "Gallery" not in credit and "Charitable" not in credit:
        x = modified_credit.replace(a_string, "").split(" and ")
        if " " in x[0] and " " in x[0]:
            f = x[0]
            s = x[1]
            add_agent(idx, f)
            add_agent(idx, s)
        else: 
            add_agent(idx, modified_credit.replace(a_string, ""))
    else:
        add_agent(idx, modified_credit.replace(a_string, ""))

    


a = 0
while a < 2:
    for idx, row in italianDataset.iterrows():
        if row["Credit Line"].count(";") > 1:
            italianDataset.at[idx, "Credit Line"]=row["Credit Line"].replace(";", ",")
        if row["Credit Line"].startswith("The Metropolitan Museum of Art, New York."):
            italianDataset.at[idx, "Credit Line"] = row["Credit Line"].replace("The Metropolitan Museum of Art, New York. ", "")
        if ":" in row["Credit Line"]:
            x = row["Credit Line"]
            x = x.replace(":",  ",")
            italianDataset.at[idx, "Credit Line"] = x
        if ", by exchange," in row["Credit Line"]:
            italianDataset.at[idx, "Credit Line"] = row["Credit Line"].replace(", by exchange",  "")
    a = a + 1
italianDataset["lastTransfer"] = empty
italianDataset["agentsTransfer"] = resp
for idx, row in italianDataset.iterrows():        
    if ";" not in row["Credit Line"]:
        credit_line = row["Credit Line"].split(", ")
        modified_credit = credit_line[0]
        if modified_credit == "Anonymous Gift" and len(credit_line) == 2 or row["Credit Line"].startswith("Anonymous Gift, in memory"):
            add_last_transfer(idx, "Gift")
        if modified_credit.startswith("Bequest of "):
            add_last_transfer(idx, "Bequest")
            if " in honor of" in modified_credit:
                modified_credit = modified_credit.split(" in honor of")[0]
            and_case("Bequest of ", modified_credit)
        elif modified_credit.startswith("The Bequest of "):
            add_last_transfer(idx, "Bequest")
            and_case("The Bequest of ", modified_credit)
        elif modified_credit.startswith("The Collection of ") or modified_credit.startswith("From the Collection of "):
            if modified_credit.startswith("From the Collection of "):
                modified_credit = modified_credit.replace ("From ", "")
            add_agent(idx, modified_credit)
            add_last_transfer(idx, "Collection")
        elif modified_credit.startswith("Gift of "):
            add_last_transfer(idx, "Gift")
            and_case("Gift of ", modified_credit)
            for n in range(len(credit_line)-2):
                n = n + 1
                if "Purchase" not in credit_line[n] and "Collection" not in credit_line[n] and "Bequest" not in credit_line[n] and "Fund" not in credit_line[n] and "Gift" not in credit_line[n] and "memory" not in credit_line[n] and "in honor" not in credit_line[n] and "in celebration" not in credit_line[n]:
                    if "in memory of h" not in credit_line[n-1] and "in honor" not in credit_line[n-1]:
                        add_agent(idx, credit_line[n].replace("and ", "", 1))
        elif modified_credit.startswith("Purchase"):
            add_last_transfer(idx, "Purchase")
        elif modified_credit.endswith("Collection") or modified_credit.endswith("Gallery"):
            add_agent(idx, modified_credit)
            add_last_transfer(idx, "Collection")
        elif modified_credit.endswith("Fund") or modified_credit.endswith("Trust"):
            add_agent(idx, modified_credit)
            add_last_transfer(idx, "Purchase")
        elif modified_credit.endswith("Funds") and " and " in modified_credit:
            add_last_transfer(idx, "Purchase")
            for el in modified_credit.split(" and "):
                add_agent(idx, (el + " Fund"). replace(" Funds", ""))
        elif "Funds" in row["Credit Line"] and "Bequest" not in row["Credit Line"] and "Purchase" not in row["Credit Line"] and "Gift" not in row["Credit Line"] and "funds from various donors" not in row["Credit Line"]:
            add_last_transfer(idx, "Purchase")
            split = row["Credit Line"].split(",")
            if len(split) > 2:
                for n in range(len(split)-1):
                    a = split[n].replace(" Funds", "") + " Fund"
                    if a.startswith(" and"):
                        a =a.replace(" and ", "")
                    elif a.startswith(" "):
                        a =a.replace (" ", "", 1)
                    add_agent(idx, a)
            else:
                if split[0] == "Funds from various donors":
                    italianDataset.at[idx, "agentsTransfer"] = italianDataset.at[idx, "agentsTransfer"]
                else:
                    new_split = split[0].split(" and ")
                    for n in range(len(new_split)):
                        a = new_split[n].replace(" Funds", "") + " Fund"
                        add_agent(idx, a)       
        if len(credit_line) > 2:
            for n in range(len(credit_line)-2):
                num = n + 1
                if credit_line[num] and "in memory " not in credit_line[num]:
                    new_modified_credit = credit_line[num]
                    if new_modified_credit.startswith("Bequest and Gift"):
                        add_agent(idx, new_modified_credit.replace("Bequest and Gift of ", ""))
                    if new_modified_credit.startswith("Bequest of"):
                        if " and " in new_modified_credit and "Mr" not in new_modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit:
                            if " and Gift " in new_modified_credit:
                                x = new_modified_credit.replace("Bequest of ", "").split(" and Gift of ")
                                add_agent(idx, x[0])
                                add_agent(idx, x[1])
                            else:
                                x = new_modified_credit.replace("Bequest of ", "").split(" and ")
                                if " " in x[0] and " " in x[0]:
                                    f = x[0]
                                    s = x[1]
                                    add_agent(idx, f)
                                    add_agent(idx, s)
                                else: 
                                    add_agent(idx, new_modified_credit.replace(" Bequest of ", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("Bequest of ", ""))
                    if new_modified_credit.startswith("Bequests of"):
                        if " and " in new_modified_credit and "Mr" not in new_modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit:
                            x = new_modified_credit.replace("Bequests of ", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                add_agent(idx, new_modified_credit.replace("Bequest of ", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("Bequests of ", ""))
                        new_n = num + 1
                        while new_n != 0:
                            if not credit_line[new_n].startswith("1") and "Bequest" not in credit_line[new_n] and "Gift" not in credit_line[new_n] and "Purchase" not in credit_line[new_n] and "Fund" not in credit_line[new_n] and "Collection" not in credit_line[new_n]:
                                if "in memory " not in credit_line[new_n] and "in memory of h" not in credit_line[new_n-1]:
                                    add_agent(idx, credit_line[new_n])
                                new_n = new_n + 1
                            else:
                                new_n = 0
                    if new_modified_credit.startswith("and Gifts of"):
                        if " and " in new_modified_credit:
                            x = new_modified_credit.split(" and ")
                            for el in x:
                                add_agent(idx, el.replace("and Gifts of ", ""))
                    if new_modified_credit.startswith("and Bequests of"):
                        if " and " in new_modified_credit and "Mr" not in new_modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit:
                            x = new_modified_credit.replace("and Bequests of ", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                add_agent(idx, new_modified_credit.replace(" Bequest of ", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("and Bequests of ", ""))
                    if new_modified_credit.startswith("and Bequest of"):
                        add_agent(idx, new_modified_credit.replace("and Bequest of ", ""))
                    if new_modified_credit.startswith("Gift of"):
                        if " and " in new_modified_credit and "Mr" not in modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit and "and Bequest" not in new_modified_credit:
                            x = new_modified_credit.replace("Gift of ", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                add_agent(idx, new_modified_credit.replace(" Gift of ", ""))
                        elif " and Bequest of " in new_modified_credit:
                            if new_modified_credit.startswith("Gift and"):
                                new_modified_credit = new_modified_credit.replace("Gift", "")
                            x = new_modified_credit.split(" and Bequest of ")
                            for el in x:
                                add_agent(idx, el.replace("Gift of ", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("Gift of ", ""))
                    if new_modified_credit.startswith("and Gift "):
                        add_agent(idx, new_modified_credit.replace("and Gift of ", ""))
                    if new_modified_credit.endswith("Gift"):
                        if " and " in new_modified_credit and "Mr" not in modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit and "and Bequest" not in new_modified_credit:
                            x = new_modified_credit.replace("Gift", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                if new_modified_credit.replace("Gift", "").startswith(" and"):
                                    add_agent(idx, new_modified_credit.replace("Gift", "").replace("and ", ""))
                                else:
                                    add_agent(idx, new_modified_credit.replace("Gift", ""))
                        elif "Fund" in new_modified_credit and " and " in new_modified_credit:
                            x = new_modified_credit.split(" and ")
                            for el in x:
                                add_agent(idx, el.replace(" Gift", "")) 
                        else:
                            a = new_modified_credit.replace(" Gift", "")
                            if a.startswith("and "):                       
                                a=a.replace("and ", "", 1)
                            add_agent(idx, a)
                    if new_modified_credit.endswith("Bequest"):
                        if " and " in new_modified_credit and "Mr" not in modified_credit and "Mrs." not in new_modified_credit and "Foundation" not in new_modified_credit and "Fund" not in new_modified_credit and "Collection" not in new_modified_credit and "Gallery" not in new_modified_credit and "Charitable" not in new_modified_credit and "and Bequest" not in new_modified_credit:
                            x = new_modified_credit.replace("Bequest", "").split(" and ")
                            if " " in x[0] and " " in x[0]:
                                f = x[0]
                                s = x[1]
                                add_agent(idx, f)
                                add_agent(idx, s) 
                            else: 
                                add_agent(idx, new_modified_credit.replace("Gift", ""))
                        else:
                            add_agent(idx, new_modified_credit.replace("Bequest", ""))
                    if new_modified_credit.endswith("Fund"):
                        if "Gift" in new_modified_credit and " and " in new_modified_credit and "and Bequest of" not in new_modified_credit:
                            x = new_modified_credit.split(" and ")
                            for el in x:
                                add_agent(idx, el.replace(" Gift", "")) 
                        elif "Gift and Bequest of" in new_modified_credit:
                            x = new_modified_credit.replace("Gift and Bequest of ", "").split(" and ")
                            for el in x:
                                add_agent(idx, el)
                        else:
                            add_agent(idx, new_modified_credit)
                    if new_modified_credit.endswith("Funds"):
                        y = num
                        if "Purchase" not in credit_line[y-1] and "Gift" not in credit_line[y-1] and "Bequest" not in credit_line[y-1] and "Collection" not in credit_line[y-1]:
                            add_agent(idx, new_modified_credit.replace(" Funds", " Fund").replace("and ", "", 1))
                            
                            while y > 1:
                                y = y - 1
                                if "Purchase" not in credit_line[y] and "Gift" not in credit_line[y] and "Bequest" not in credit_line[y] and "Collection" not in credit_line[y] and "in memory" not in credit_line[y]:
                                    if credit_line[y].startswith("and "):
                                        add_agent(idx, (credit_line[y] + " Fund").replace("and ", "", 1))
                                    else:
                                        add_agent(idx, (credit_line[y] + " Fund"))
                                else:
                                    y = 0
                        else:
                            if " and " in new_modified_credit:
                                for el in new_modified_credit.split(" and "):
                                    add_agent(idx, (el.replace(" Funds", "") + " Fund"))
                    if new_modified_credit.endswith(" Gifts"):
                        
                        if new_modified_credit.count(" and ") > 1 and not new_modified_credit.startswith(" and "):
                            x = new_modified_credit.split(" and ")
                            for n in range(len(x)-1):
                                num = n + 1
                                if " " not in x[num-1] and " " in x[num]:
                                    f = x[num-1] + " and " + x[num]
                                    add_agent(idx, f.replace(" Gifts", ""))
                                else:
                                    add_agent(idx, x[num].replace(" Gifts", ""))
                        elif "Foundation" in new_modified_credit and " and " in new_modified_credit:
                            x = new_modified_credit.split(" and ")
                            for el in x:
                                add_agent(idx, el.replace(" Gifts", ""))
                        else:
                            y = num
                            if new_modified_credit.startswith("and "):
                                new_modified_credit = new_modified_credit.replace("and ", "", 1)
                            add_agent(idx, new_modified_credit.replace(" Gifts", ""))
                            while y > 1:
                                y = y - 1
                                if "Purchase" not in credit_line[y] and "Gift" not in credit_line[y] and "Bequest" not in credit_line[y] and "Collection" not in credit_line[y]:
                                    add_agent(idx, credit_line[y])
                                else: 
                                    y = 0
                                
    elif row["Credit Line"].count(";") == 1:
        credit_line_update = row["Credit Line"].split(";")
        credit_line_segment_one = credit_line_update[0].split(",")
        if credit_line_segment_one[0].startswith("Gift of"):
            add_last_transfer(idx, "Gift")
            if " and " in credit_line_segment_one[0] and "Mr" not in credit_line_segment_one[0] and "Mrs." not in credit_line_segment_one[0] and "Foundation" not in credit_line_segment_one[0] and "Fund" not in credit_line_segment_one[0] and "Collection" not in credit_line_segment_one[0] and "Gallery" not in credit_line_segment_one[0] and "Charitable" not in credit_line_segment_one[0]:
                x = credit_line_segment_one[0].replace("Gift of ", "").split(" and ")
                if " " in x[0] and " " in x[0]:
                    f = x[0]
                    s = x[1]
                    add_agent(idx, f)
                    add_agent(idx, s) 
                else: 
                    add_agent(idx, credit_line_segment_one[0].replace("Gift of ", ""))
            else:
                add_agent(idx, credit_line_segment_one[0].replace("Gift of ", ""))
        if credit_line_segment_one[0].startswith("Purchase"):
            add_last_transfer(idx, "Purchase")
        if credit_line_segment_one[-1].endswith("Gifts"):

            for n in range(len(credit_line_segment_one) - 1):
                num = n + 1
                if "Collection" in credit_line_segment_one[num] or "Fund" in credit_line_segment_one[num] or "Foundation" in credit_line_segment_one[num]:
                    add_agent(idx, credit_line_segment_one[num].replace(" ", "", 1))
                else:
                    if credit_line_segment_one[num].startswith(" and"):
                        credit_line_segment_one[num] = credit_line_segment_one[num].replace(" and", "")
                    add_agent(idx, credit_line_segment_one[num].replace(" ", "", 1).replace(" Gifts", ""))
        credit_line_segment_two = credit_line_update[1].split(",")
        if credit_line_segment_two[0].startswith(" and Gifts"): 
            for n in range(len(credit_line_segment_two) - 1):
                num = n
                if "Bequest" not in credit_line_segment_two[num] and "Collection" not in credit_line_segment_two[num] and "Fund" not in credit_line_segment_two[num] and "by exchange" not in credit_line_segment_two[num]:
                    if "Mr. and Mrs." in credit_line_segment_two[num]:
                        add_agent(idx, credit_line_segment_two[num].replace(" and Gifts of","").replace(" ", "", 1))
                    else:
                        add_agent(idx, credit_line_segment_two[num].replace(" and Gifts of","").replace(" and ", " ", 1).replace(" ", "", 1))
                elif "Collection" in credit_line_segment_two[num] or "Fund" in credit_line_segment_two[num]:
                    add_agent(idx, credit_line_segment_two[num].replace(" and","").replace(" ", "", 1))
                elif "Bequest" in credit_line_segment_two[num]:
                    if credit_line_segment_two[num].startswith(" Bequests of"):
                        inner_list = credit_line_segment_two[num].split(" and")
                        for el in inner_list:
                            add_agent(idx, el.replace(" Bequests of","").replace(" ", "", 1))
        for el in credit_line_segment_two:
            if "Collection" in el:
                add_agent(idx, el.replace(" ", "", 1))

#private collections are often related to the bequest of a private owner, we thus need to associated both the collection and the owner with a unique entity 
for idx, row in italianDataset.iterrows():    
    dealer = row["agentsTransfer"]
    if "Collection," in str(dealer):
        dealerList = str(dealer).split(", ")
        italianDataset.at[idx, "agentsTransfer"] = dealerList[1]


italianDataset.to_csv("data_management/transfers.csv")

italianDataset

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,...,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL,lastTransfer,agentsTransfer
0,1985.5,0,0,1,435573,None,European Paintings,1985.0,Painting,Flora and Zephyr,...,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912099,None,"Metropolitan Museum of Art, New York, NY",Goddess|Putti|Flowers|Landscapes,http://vocab.getty.edu/page/aat/300343852|http...,https://www.wikidata.org/wiki/Q205985|https://...,Purchase,Rudolph and Lentilhon G. von Fluegge Foundatio...
1,12.6,0,0,1,435574,None,European Paintings,1912.0,"Painting, predella panel",The Crucifixion,...,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905919,None,"Metropolitan Museum of Art, New York, NY",Soldiers|Men|Crucifixion|Horses|Mountains|Ange...,http://vocab.getty.edu/page/aat/300185678|http...,https://www.wikidata.org/wiki/Q4991371|https:/...,Purchase,Rogers Fund
2,42.53.2,0,0,1,435575,None,European Paintings,1942.0,"Painting, miniature","Jérôme Bonaparte (1784–1860), King of Westphalia",...,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19913805,None,"Metropolitan Museum of Art, New York, NY",Kings|Men|Portraits,http://vocab.getty.edu/page/aat/300025481|http...,https://www.wikidata.org/wiki/Q12097|https://w...,Gift,Helen O. Brice
3,1991.27.2,0,1,1,435576,624.0,European Paintings,1991.0,Painting,Saint Alexander,...,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912513,None,"Metropolitan Museum of Art, New York, NY",Bishops|Croziers|Miter,http://vocab.getty.edu/page/aat/300025756|http...,https://www.wikidata.org/wiki/Q29182|https://w...,Bequest,Lucy G. Moses
4,14.40.628,0,1,1,435577,None,European Paintings,1914.0,Painting,The Crucifixion,...,None,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905200,None,"Metropolitan Museum of Art, New York, NY",Crucifixion|Saints|Mary Magdalene|Saint Franci...,http://vocab.getty.edu/page/aat/300404300|http...,https://www.wikidata.org/wiki/Q3235597|https:/...,Bequest,Benjamin Altman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,2020.263.5,0,0,1,816624,627.0,European Paintings,2020.0,Painting,Basket of Fruit,...,None,http://www.metmuseum.org/art/collection/search...,None,None,"Metropolitan Museum of Art, New York, NY",Grapes|Fruit|Still Life|Apples|Pomegranates,http://vocab.getty.edu/page/aat/300379338|http...,https://www.wikidata.org/wiki/Q10978|https://w...,Bequest,Errol M. Rudman
528,2020.263.8,0,0,1,816629,623.0,European Paintings,2020.0,Painting,The Flight into Egypt,...,None,http://www.metmuseum.org/art/collection/search...,None,None,"Metropolitan Museum of Art, New York, NY",None,None,None,Bequest,Errol M. Rudman
529,2020.263.6,0,0,1,816630,None,European Paintings,2020.0,Painting,Charity,...,None,http://www.metmuseum.org/art/collection/search...,None,None,"Metropolitan Museum of Art, New York, NY",Nursing|Sleeping|Female Nudes|Infants,http://vocab.getty.edu/page/aat/300379905|http...,https://www.wikidata.org/wiki/Q174876|https://...,Bequest,Errol M. Rudman
530,2020.112,0,0,0,817124,618.0,European Paintings,2020.0,Painting,Cain Slaying Abel,...,None,http://www.metmuseum.org/art/collection/search...,None,None,"Metropolitan Museum of Art, New York, NY",Abel|Cain,http://vocab.getty.edu/page/ia/901000498|http:...,https://www.wikidata.org/wiki/Q313421|https://...,Gift,Alan M. May
